In [ ]:
import os
import sys

sys.path.append("../../../")

from config import *
import wandb
import torch
from typing import Dict
from losses.losses import build_loss_fn
from optimizers.optimizers import build_optimizer
from train_scripts.train_functions import train_val
from architectures.leavisnet_v3 import build_model

# from datasets.build_dataset import build_architecture, build_dataloader

from collections import namedtuple
from torch.utils.data import DataLoader
import monai

In [ ]:
model = build_test_model(None)
with torch.cuda.device(0):
    net = model
    macs, params = get_model_complexity_info(
        net, (3, 224, 224), as_strings=True, print_per_layer_stat=True, verbose=True
    )
    print("{:<30}  {:<8}".format("Computational complexity: ", macs))
    print("{:<30}  {:<8}".format("Number of parameters: ", params))

In [ ]:
import torch
from torch.profiler import profile, record_function, ProfilerActivity

In [ ]:
model = build_test_model(None).cpu()
inputs = torch.randn(1, 3, 224, 224)

pytorch_total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("\nmodel parameter count = ", pytorch_total_params)

In [ ]:
def estimate_memory_inference(
    model, sample_input, batch_size=1, use_amp=False, device=0
):
    """Predict the maximum memory usage of the model.
    Args:
        optimizer_type (Type): the class name of the optimizer to instantiate
        model (nn.Module): the neural network model
        sample_input (torch.Tensor): A sample input to the network. It should be
            a single item, not a batch, and it will be replicated batch_size times.
        batch_size (int): the batch size
        use_amp (bool): whether to estimate based on using mixed precision
        device (torch.device): the device to use
    """
    # Reset model and optimizer
    model.cpu()
    a = torch.cuda.memory_allocated(device)
    model.to(device)
    b = torch.cuda.memory_allocated(device)
    model_memory = b - a
    model_input = sample_input  # .unsqueeze(0).repeat(batch_size, 1)
    output = model(model_input.to(device)).sum()
    total_memory = model_memory

    return total_memory


estimate_memory_inference(model, inputs)

In [ ]:
model.cpu()
with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cpu_time_total", row_limit=10))

In [ ]:
model = model.cuda()
inputs = torch.randn(1, 3, 224, 224).cuda()

with profile(
    activities=[ProfilerActivity.CPU, ProfilerActivity.CUDA], record_shapes=True
) as prof:
    with record_function("model_inference"):
        model(inputs)

print(prof.key_averages().table(sort_by="cuda_time_total", row_limit=10))

In [ ]:
import torch
import matplotlib.pyplot as plt

model = torch.nn.Linear(2, 1)
optimizer = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=20, T_mult=2, eta_min=0.001, last_epoch=-1
)


lrs = []

for i in range(1000):
    lr_sched.step()
    lrs.append(optimizer.param_groups[0]["lr"])

plt.plot(lrs)